In [1]:
import numpy as np
import os
import nibabel as nib
import shutil
from PIL import Image
import random
import cv2

In [2]:
classes = ['MCI', 'NC']


def mkdir(path):
    floder = os.path.exists(path)
    if not floder:
        os.makedirs(path)
    else:
        pass


for i in classes:
    file = './PET/val/{}'.format(i)
    mkdir(file)

In [3]:
def generate_val(t_dir, v_dir, p=0.15):
    # 从训练集中抽取一部分当作测试集
    pathDir = os.listdir(t_dir)
    sample = random.sample(pathDir, int(len(pathDir) * p))
    for item in sample:
        shutil.move(t_dir + '\\' + item, v_dir + '\\' + item)
    return

In [4]:
for i in classes:
    train_dir = './PET/train/{}'.format(i)
    val_dir = './PET/val/{}'.format(i)
    generate_val(train_dir, val_dir)

In [5]:
def join_train(source, target):
    # 用于将测试集移回训练集
    pathDir = os.listdir(source)
    for item in pathDir:
        shutil.move(source + '\\' + item, target + '\\' + item)
    return

In [6]:
for i in classes:
    train_dir = './PET/train/{}'.format(i)
    val_dir = './PET/val/{}'.format(i)
    join_train(val_dir, train_dir)

In [6]:
def nii_to_jpg(source, target):
    # 将source文件夹下面的nii图片全部转为jpg格式，保存在target文件夹下
    items = os.listdir(source)
    for item in items:
        img_sou = os.path.join(source, item)
        img = nib.load(img_sou)
        img_data = img.get_fdata()
        img_data = img_data[:, :, :, 0]
        name = item.replace('.nii', '')
        img_tar = os.path.join(target, name)
        mkdir(img_tar)

        (x, y, z) = img_data.shape
        for i in range(int(z / 8), int(7 * z / 8)):
            slice = img_data[:, :, i]
            slice_min = np.min(slice)
            slice_max = np.max(slice)
            # 归一化处理
            slice_normalized = (slice - slice_min) / (slice_max - slice_min)
            slice_scaled = (slice_normalized * 255).astype(np.uint8)
            slice_image = Image.fromarray(slice_scaled)
            slice_image.save(os.path.join(img_tar, '{}.jpg'.format(i)))


In [7]:
base = ['./PET/train/MCI',
        './PET/train/NC',
        './PET/val/MCI',
        './PET/val/NC',
        './PET/test']
output = ['./PETjpg/train/MCI',
          './PETjpg/train/NC',
          './PETjpg/val/MCI',
          './PETjpg/val/NC',
          './PETjpg/test']
for i in output:
    mkdir(i)
for i, j in zip(base, output):
    nii_to_jpg(i, j)

In [8]:
base = ['PET/train/MCI',
        'PET/train/NC',
        'PET/val/MCI',
        'PET/val/NC',
        'PET/test']
output = ['PETjpg/train/MCI',
          'PETjpg/train/NC',
          'PETjpg/val/MCI',
          'PETjpg/val/NC',
          'PETjpg/test']
os.chdir('E:/code/pycharm/MLhw/datawhale/CV/')
for cur_dir in output:
    # cur_dirs即是01，02，03的文件夹集合
    cur_dirs = os.listdir(cur_dir)
    for folder in cur_dirs:
        # folder代表的是样品编号，即从01-25
        folder_dir = os.listdir('E:/code/pycharm/MLhw/datawhale/CV/' + cur_dir + '/' + folder)
        os.chdir('E:/code/pycharm/MLhw/datawhale/CV/' + cur_dir + '/' + folder)
        for img_num in folder_dir:
            os.rename(img_num, folder + '_' + img_num)
            # 改名为x_y，表示x号样本的y号切片
            shutil.move('E:/code/pycharm/MLhw/datawhale/CV/' + cur_dir + '/' + folder + '/' + folder + '_' + img_num,
                        'E:/code/pycharm/MLhw/datawhale/CV/' + cur_dir + '/')
    os.chdir('E:/code/pycharm/MLhw/datawhale/CV/')
    for folder in cur_dirs:
        shutil.rmtree('E:/code/pycharm/MLhw/datawhale/CV/' + cur_dir + '/' + folder)

In [9]:
for i in classes:
    train_dir = './PETjpg/train/{}'.format(i)
    files = os.listdir(train_dir)
    print(len(files))

936
1616


In [12]:
def group_images_by_id(source_folder, target_folder):
    # 创建目标文件夹
    os.makedirs(target_folder, exist_ok=True)

    # 遍历源文件夹中的所有图片
    for filename in os.listdir(source_folder):
        if filename.endswith('.jpg'):
            # 提取图片的组号x和组内编号y
            x, y = filename.split('_')
            group_folder = os.path.join(target_folder, x, 'NC')

            # 如果该组的文件夹不存在，则创建该组的文件夹
            os.makedirs(group_folder, exist_ok=True)

            # 将图片移动到对应的组文件夹中
            src_path = os.path.join(source_folder, filename)
            dest_path = os.path.join(group_folder, filename)
            shutil.copy(src_path, dest_path)


source_folder = './PETjpg/test/NC'
target_folder = './PETjpg/test1'
group_images_by_id(source_folder, target_folder)